In [10]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display


ModuleNotFoundError: No module named 'helper_code'

In [11]:
import sys
sys.path.insert(0,'/home/ubuntu/hmd/notebooks')
sys.path.insert(0,'/home/ubuntu/hmd/iy_classifier')


In [13]:
from helper_code import *
from utils.get_feature import *

In [5]:
root_dir = '/home/ubuntu/hmd/notebooks/physionet.org/files/circor-heart-sound/1.0.1'
training_data_file = root_dir + '/' + 'training_data.csv'
training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [6]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Wed Mar 23 16:19:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:05.0 Off |                  Off |
| N/A   32C    P0    37W / 300W |      3MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:00:06.0 Off |                  Off |
| N/A   

In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [9]:
df = pd.read_csv(training_data_file)
df[:10]

,Patient ID,Locations,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur grading,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.90,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.10,False,Present,AV+MV+PV+TV,TV,...,III/VI,High,Harsh,NaN,NaN,NaN,NaN,NaN,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.10,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.90,False,Present,TV,TV,...,I/VI,Low,Blowing,NaN,NaN,NaN,NaN,NaN,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.20,False,Present,AV+MV+PV+TV,PV,...,II/VI,Low,Harsh,NaN,NaN,NaN,NaN,NaN,CC2015,NaN
5,14998,AV+PV+TV+MV,Child,Male,NaN,NaN,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC2015,NaN
6,23625,AV+PV+TV+MV,Child,Female,92.0,14.00,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC2015,50379.0
7,24160,AV+PV+TV+MV,Child,Female,98.0,17.66,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC2015,NaN
8,29045,AV+PV+TV+MV,Child,Female,88.0,12.50,False,Present,AV+MV+PV+TV,TV,...,II/VI,Low,Blowing,NaN,NaN,NaN,NaN,NaN,CC2015,NaN
9,29378,AV+MV,Child,Female,82.0,10.70,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CC2015,NaN


In [14]:
data_folder =  '/home/ubuntu/hmd/notebooks/physionet.org/files/circor-heart-sound/1.0.1/training_data'
model_folder = 'tmp_model1'

In [15]:
data_folder

'/home/ubuntu/hmd/notebooks/physionet.org/files/circor-heart-sound/1.0.1/training_data'

In [16]:
patient_files = find_patient_files(data_folder)
num_patient_files = len(patient_files)


In [18]:
len(patient_files)

942

In [22]:
shuffled_idx = np.random.permutation(942)

In [25]:
patient_files_trn = patient_files[:800]
patient_files_test = patient_files[800:]
num_patient_files = len(patient_files_trn)

In [24]:
os.makedirs(model_folder, exist_ok=True)

classes = ['Present', 'Unknown', 'Absent']
num_classes = len(classes)


In [28]:
i = 0
current_patient_data = load_patient_data(patient_files_trn[i])

In [29]:
current_patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Campaign: CC2015\n#Additional ID: nan'

In [ ]:
# Load recordings. wav data recording
def load_recordings(data_folder, data, get_frequencies=False):
    num_locations = get_num_locations(data)
    recording_information = data.split('\n')[1:num_locations+1]

    recordings = list()
    frequencies = list()
    for i in range(num_locations):
        entries = recording_information[i].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)
        recording, frequency = load_wav_file(filename)
        recordings.append(recording)
        frequencies.append(frequency)

    if get_frequencies:
        return recordings, frequencies
    else:
        return recordings

In [32]:
num_locations = get_num_locations(current_patient_data)
recording_information = current_patient_data.split('\n')[1:num_locations+1]

In [34]:
print(num_locations, recording_information)


4 ['AV 2530_AV.hea 2530_AV.wav 2530_AV.tsv', 'PV 2530_PV.hea 2530_PV.wav 2530_PV.tsv', 'TV 2530_TV.hea 2530_TV.wav 2530_TV.tsv', 'MV 2530_MV.hea 2530_MV.wav 2530_MV.tsv']


In [137]:
recording_information

['AV 2530_AV.hea 2530_AV.wav 2530_AV.tsv',
 'PV 2530_PV.hea 2530_PV.wav 2530_PV.tsv',
 'TV 2530_TV.hea 2530_TV.wav 2530_TV.tsv',
 'MV 2530_MV.hea 2530_MV.wav 2530_MV.tsv']

In [35]:
entries = recording_information[i].split(' ')
entries

['AV', '2530_AV.hea', '2530_AV.wav', '2530_AV.tsv']

In [36]:
recording_file = entries[2]

In [37]:
recording_file

'2530_AV.wav'

In [38]:
filename = os.path.join(data_folder, recording_file)
filename

'/home/ubuntu/hmd/notebooks/physionet.org/files/circor-heart-sound/1.0.1/training_data/2530_AV.wav'

In [107]:
features = dict()

In [108]:
features = dict()
features['id'] = []
features['age'] = []
features['sex'] = []
features['hw'] = []
features['preg'] = []
features['loc'] = []
features['mel1'] = []


In [110]:
id1 = recording_file.split('_')[0]
features['id'].append(id1)

In [111]:
mel1 = feature_extract_melspec(filename)[0]
mel1.shape

(100, 313)

In [112]:
features['mel1'].append(mel1)

In [113]:
import tensorflow as tf

In [114]:
from tensorflow.keras.utils import to_categorical

In [115]:
entries_age = ['Child']
age_group = get_age(current_patient_data)

In [116]:
age_group

'Child'

In [117]:
age_group = get_age(current_patient_data)

age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
current_age_group = np.zeros(6, dtype=int)

if age_group in age_classes:
    j = age_classes.index(age_group)
    current_age_group[j] = 1
else :
    current_age_group[5] = 1



In [118]:
age_group

'Child'

In [119]:
current_age_group

array([0, 0, 1, 0, 0, 0])

In [120]:
age_group = 'ddd'

age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
current_age_group = np.zeros(6, dtype=int)

if age_group in age_classes:
    j = age_classes.index(age_group)
    current_age_group[j] = 1
else :
    current_age_group[5] = 1

In [121]:
current_age_group

array([0, 0, 0, 0, 0, 1])

In [122]:
features['age'].append(current_age_group)

In [123]:
# Extract sex. Use one-hot encoding.
sex = get_sex(current_patient_data)
sex_features = np.zeros(2, dtype=int)
if compare_strings(sex, 'Female'):
    sex_features[0] = 1
elif compare_strings(sex, 'Male'):
    sex_features[1] = 1

In [124]:
features['sex'].append(sex_features)

In [125]:
# Extract height and weight.
height = get_height(current_patient_data)
weight = get_weight(current_patient_data)
features['hw'].append(np.array([height, weight]))

In [126]:
is_pregnant = get_pregnancy_status(current_patient_data)
features['preg'].append(is_pregnant)

In [127]:
locations = entries[0]

recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
num_recording_locations = len(recording_locations)
loc_features = np.zeros(num_recording_locations)
if locations in recording_locations:
    j = recording_locations.index(locations)
    loc_features[j] = 1

In [128]:
locations

'AV'

In [129]:
loc_features

array([1., 0., 0., 0., 0.])

In [130]:
features['loc'].append(loc_features)

In [131]:
features

{'id': ['2530'],
 'age': [array([0, 0, 0, 0, 0, 1])],
 'sex': [array([1, 0])],
 'hw': [array([98. , 15.9])],
 'preg': [False],
 'loc': [array([1., 0., 0., 0., 0.])],
 'mel1': [array([[-14.24342  , -14.460995 , -12.48743  , ...,  -8.350793 ,
           -9.550913 ,  -6.6099434],
         [ -8.833427 , -11.78526  , -10.376642 , ..., -10.638776 ,
          -14.766047 ,  -2.595482 ],
         [ -7.201795 , -11.942595 , -14.396763 , ..., -13.780443 ,
          -11.831632 , -10.3140745],
         ...,
         [-43.652073 , -60.107727 , -58.363945 , ..., -46.318718 ,
          -47.93747  , -47.206367 ],
         [-42.097    , -59.548874 , -57.106033 , ..., -45.140682 ,
          -53.84076  , -53.963337 ],
         [-42.240356 , -60.107727 , -60.107727 , ..., -51.811592 ,
          -55.905552 , -57.170105 ]], dtype=float32)]}

In [ ]:

recording_information = current_patient_data.split('\n')[1:num_locations+1]

In [132]:

    current_patient_data = load_patient_data(patient_files[i])
    current_recordings = load_recordings(data_folder, current_patient_data)

In [135]:
current_patient_data

'2530 4 4000\nAV 2530_AV.hea 2530_AV.wav 2530_AV.tsv\nPV 2530_PV.hea 2530_PV.wav 2530_PV.tsv\nTV 2530_TV.hea 2530_TV.wav 2530_TV.tsv\nMV 2530_MV.hea 2530_MV.wav 2530_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 98.0\n#Weight: 15.9\n#Pregnancy status: False\n#Murmur: Absent\n#Murmur locations: nan\n#Most audible location: nan\n#Systolic murmur timing: nan\n#Systolic murmur shape: nan\n#Systolic murmur grading: nan\n#Systolic murmur pitch: nan\n#Systolic murmur quality: nan\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Campaign: CC2015\n#Additional ID: nan'

In [136]:
num_patient_files

800

In [143]:
features = dict()
features['id'] = []
features['age'] = []
features['sex'] = []
features['hw'] = []
features['preg'] = []
features['loc'] = []
features['mel1'] = []
labels = []

age_classes = ['Neonate', 'Infant', 'Child', 'Adolescent', 'Young Adult']
recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']

for i in range(num_patient_files):

    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files_trn[i])
    num_locations = get_num_locations(current_patient_data)
    recording_information = current_patient_data.split('\n')[1:num_locations+1]
    for j in range(num_locations) :
        entries = recording_information[j].split(' ')
        recording_file = entries[2]
        filename = os.path.join(data_folder, recording_file)
        
        # Extract id
        id1 = recording_file.split('_')[0]
        features['id'].append(id1)
        
        # Extract melspec
        mel1 = feature_extract_melspec(filename)[0]
        features['mel1'].append(mel1)
        
        # Extract age_group
        age_group = get_age(current_patient_data)
        current_age_group = np.zeros(6, dtype=int)
        if age_group in age_classes:
            j = age_classes.index(age_group)
            current_age_group[j] = 1
        else :
            current_age_group[5] = 1
        features['age'].append(current_age_group)
        
        # Extract sex
        sex = get_sex(current_patient_data)
        sex_features = np.zeros(2, dtype=int)
        if compare_strings(sex, 'Female'):
            sex_features[0] = 1
        elif compare_strings(sex, 'Male'):
            sex_features[1] = 1
        features['sex'].append(sex_features)
        
        # Extract height and weight.
        height = get_height(current_patient_data)
        weight = get_weight(current_patient_data)
        features['hw'].append(np.array([height, weight]))
        
        # Extract pregnancy
        is_pregnant = get_pregnancy_status(current_patient_data)
        features['preg'].append(is_pregnant)

        # Extract location
        locations = entries[0]
        num_recording_locations = len(recording_locations)
        loc_features = np.zeros(num_recording_locations)
        if locations in recording_locations:
            j = recording_locations.index(locations)
            loc_features[j] = 1
        features['loc'].append(loc_features)

        # Extract labels and use one-hot encoding.
        current_labels = np.zeros(num_classes, dtype=int)
        label = get_label(current_patient_data)
        if label in classes:
            j = classes.index(label)
            current_labels[j] = 1
        labels.append(current_labels)

In [145]:
len(labels)

2675